In [ ]:
import sys
from glob import glob

In [ ]:
files = glob('../data_intensity/pkl/id_*.measurements.covd.pkl.intensityANDmorphology.csv.gz')
file = ('../data_intensity/pkl/id_13.measurements.covd.pkl.intensityANDmorphology.csv.gz')

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style="whitegrid")

In [ ]:
for f in files[:]:
    outfile = f+'.summary_stat.absolute.png'
    df = pd.read_csv(f)
    df = df.rename(columns={"cluster_intensity": "clusterID"})
    f, axes = plt.subplots(2, 3, figsize=(15, 7))
    sns.boxplot(x='clusterID', y='area', data=df, orient="v", ax=axes[0, 0])
    sns.boxplot(x='clusterID', y='perimeter', data=df, orient="v", ax=axes[0, 1])
    sns.boxplot(x='clusterID', y='solidity', data=df, orient="v", ax=axes[0, 2])
    sns.boxplot(x='clusterID', y='eccentricity', data=df, orient="v", ax=axes[1, 0])
    sns.boxplot(x='clusterID', y='mean_intensity', data=df, orient="v", ax=axes[1, 1])
    axes[-1, -1].axis('off')
    plt.tight_layout()
    plt.savefig(str(outfile))
    

In [ ]:
for f in files[:]:
    outfile = f+'.summary_stat.percentile.png'
    df = pd.read_csv(f)
    df = df.rename(columns={"cluster_intensity": "clusterID"})
    for c in df.columns[7:12]:
        df["q_"+c] = pd.qcut(df[c],100,labels=False, duplicates='drop')

    f, axes = plt.subplots(2, 3, figsize=(15, 7))
    sns.boxplot(x='clusterID', y='q_area', data=df, orient="v", ax=axes[0, 0])
    sns.boxplot(x='clusterID', y='q_perimeter', data=df, orient="v", ax=axes[0, 1])
    sns.boxplot(x='clusterID', y='q_solidity', data=df, orient="v", ax=axes[0, 2])
    sns.boxplot(x='clusterID', y='q_eccentricity', data=df, orient="v", ax=axes[1, 0])
    sns.boxplot(x='clusterID', y='q_mean_intensity', data=df, orient="v", ax=axes[1, 1])
    axes[-1, -1].axis('off')
    plt.tight_layout()
    plt.savefig(str(outfile))
    

In [ ]:
outfile = file+'.summary_stat.absolute.png'
df = pd.read_csv(file)

In [ ]:
for c in ['area', 'perimeter', 'solidity',
       'eccentricity', 'circularity', 'mean_intensity', 'std_intensity',
       'cov_intensity']:
    df["q_"+c] = pd.qcut(df[c],100,labels=False, duplicates='drop')

In [ ]:
f, axes = plt.subplots(2, 4, figsize=(15, 7))
sns.boxplot(x='clusterID1', y='area', data=df, orient="v", ax=axes[0, 0])
sns.boxplot(x='clusterID1', y='perimeter', data=df, orient="v", ax=axes[0, 1])
sns.boxplot(x='clusterID1', y='solidity', data=df, orient="v", ax=axes[0, 2])
sns.boxplot(x='clusterID1', y='eccentricity', data=df, orient="v", ax=axes[0, 3])
sns.boxplot(x='clusterID1', y='circularity', data=df, orient="v", ax=axes[1, 0])
sns.boxplot(x='clusterID1', y='mean_intensity', data=df, orient="v", ax=axes[1, 1])
sns.boxplot(x='clusterID1', y='cov_intensity', data=df, orient="v", ax=axes[1, 2])
axes[-1, -1].axis('off')
plt.tight_layout()
plt.show()
#plt.savefig(str(outfile))
    

In [ ]:
def plot_boxplot_profiles(filename):
    df = pd.read_csv(filename)
    dfmelted = pd.DataFrame()
    f, axes = plt.subplots(1, 4, figsize=(25, 10))
    idx=0
    for c in set(df["clusterID3"]):
        dfc = df[df["clusterID3"] == c][["area",
                                         "circularity",
                                         "eccentricity",
                                         "mean_intensity",
                                         "cov_intensity"]]
        data = pd.melt(dfc)
        dfmelted = dfmelted.append(data)
        chart = sns.boxplot(x='variable', y='value', data=data, 
                    meanline=True,
                    orient="v", 
                    ax=axes[idx]).set_title('Cluster ID:'+str(c+1))
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45)
        idx += 1
    plt.savefig(filename+'.boxplot.profile.png')
    plt.show()
    return

In [ ]:
for f in files[:]:
    plot_boxplot_profiles(f)

In [ ]:
for f in files[:]:
    print(f)
    df = pd.read_csv(f)
    g = sns.pairplot(df[["area","circularity","eccentricity","mean_intensity","cov_intensity"]].sample(n=10000),
                markers='.')
    g.savefig(f+'.scatter-pairplot.png')

In [ ]:
for f in files[:]:
    print(f)
    for c in set(df["clusterID3"]):
        dfc = df[df["clusterID3"] == c][["area",
                                         "circularity",
                                         "eccentricity",
                                         "mean_intensity",
                                         "cov_intensity"]]
        g = sns.pairplot(dfc.sample(n=10000),markers='.')
        g.fig.suptitle('Cluster ID: '+str(c+1), fontsize=16,y=1)
        g.savefig(f+'.scatter-pairplot.'+'clusterID_'+str(c+1)+'.png')
        plt.close()